In [ ]:
import pandas as pd
import requests
import time

In [ ]:
hotel_location_in_jakarta = r"C:/GD_2025/6_spatial_data_science/1_location_fnb/2_pipeline/location_intelligence_potential_sales_areas/data/row-data/1_input_datas/1_customer_location/hotel_location_in_jakarta.csv"

dateframe_hotel_location = pd.read_csv(hotel_location_in_jakarta)
display(dateframe_hotel_location)

In [ ]:
dataframe_cleanned = dateframe_hotel_location.drop(columns=["hotel_name_link", "review_score",	"review_score_text", "review_score_title", "hotel_image", "hotel_price"])
display(dataframe_cleanned)

In [ ]:
def get_coordinates_from_osm(location):
    osm_url = "http://overpass-api.de/api/interpreter"

    query = f"""
    [out:json];
    node["name"="{location}"];
    out center;
    """

    try:
        response = requests.get(osm_url, params={'data': query})
        data = response.json()

        if "elements" in data and len(data["elements"]) > 0:
            lat = data["elements"][0]["lat"]
            lon = data["elements"][0]["lon"]
            return lat, lon
        else:
            return None, None
    except Exception as error:
        print(f"Upss, error getting datas: {error}")

In [ ]:
dataframe_with_coordinate = dataframe_cleanned.copy()

dataframe_with_coordinate["latitude"] = None
dataframe_with_coordinate["longitude"] = None

for index, row in dataframe_with_coordinate.iterrows():
    hotel_name = row["hotel_name"]
    lat, lon = get_coordinates_from_osm(hotel_name)

    dataframe_with_coordinate.at[index, "latitude"] = lat
    dataframe_with_coordinate.at[index, "longitude"] = lon

    time.sleep(1)

print(dataframe_with_coordinate)

In [ ]:
output_get_location_customer_from_kaggle = r"C:/GD_2025/6_spatial_data_science/1_location_fnb/2_pipeline/location_intelligence_potential_sales_areas/data/row-data/2_cleanning_datas/result_customer_location_from_kaggle1.csv"

result_dataframe_with_coordinate = dataframe_with_coordinate.dropna()
result_dataframe_with_coordinate["kategori"] = "Hotel"
display(result_dataframe_with_coordinate)

result_dataframe_with_coordinate.to_csv(output_get_location_customer_from_kaggle, index=False)